Per Lily: the CPD has it's own system to flag officers they think are at high risk of doing bad things, not really sure exatly what those bad things are but that's ok. They use some metrics to flag that we don't, but use some that we do. They flag an officer with >=3 excessive use of force complaints or >=2 sustained complaints against them within a 12 month period.
This notebook is to identify officers that meet this cutoff within 2014 in our dataset, see how many of them are y=1 by our own definition

In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import sys
import os
sys.path.insert(0, os.path.abspath("../pipeline"))
import Make_By_Officer_DF as make_officer_df
import Feat_Engineering as feat_engineering_helpers
import Run_Model as run_model
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [2]:
t1 = (2014, 2014)
t2 = (2015, 2015)
additional_cont_feat, comparison_df = make_officer_df.make_df(t1, t2)


In [4]:
additional_cont_feat

['Pcnt Complaints Against ASIAN/PACIFIC ISLANDER',
 'Pcnt Complaints Against BLACK',
 'Pcnt Complaints Against HISPANIC',
 'Pcnt Complaints Against NATIVE AMERICAN/ALASKAN NATIVE',
 'Pcnt Complaints Against WHITE',
 'Pcnt Complaints Against FEMALE',
 'Pcnt Complaints Against MALE',
 'average_salary',
 'salary_change',
 'settlement',
 'trr_total',
 'Active - Firearm',
 'Active - Non-Lethal Weapon',
 'Active - Other',
 'Active - Physical Force',
 'Passive - Non-Lethal Weapon',
 'Passive - Other',
 'Passive - Physical Force',
 'Non-Violent Civilian Contact',
 'Officer Drug Use/Possession',
 'Other/Unknown',
 'Racial/Ethnic',
 'Use of Force - No Weapon',
 'Use of Force - Weapon',
 'count_sustained',
 'suspension_length',
 'officer_filed_complaints']

In [9]:
desired_targets = ['target_use_of_force', 'target_drug', 'target_racial',
                   'target_sustained', 'target_nonviolent']
target_col = "any_known_complaint"
feat_engineering_helpers.make_target_col(comparison_df, desired_targets, target_col)

In [11]:
meets_count_sustained_threshold = comparison_df["count_sustained"]>=2
meets_excessive_force_threshold = (comparison_df["Use of Force - No Weapon"]+ 
                                   comparison_df["Use of Force - Weapon"])>=3 
CPD_flagged = comparison_df[meets_count_sustained_threshold | meets_excessive_force_threshold]

In [14]:
print("Count that CPD system flags:", CPD_flagged[target_col].shape[0])
print("Number of CPD flags that are y=1:", CPD_flagged[target_col].sum())

Count that CPD system flags: 8
Number of CPD flags that are y=1: 3


In [15]:
comparison_df["count_sustained"].describe()

count    4836.000000
mean        0.051696
std         0.232373
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         3.000000
Name: count_sustained, dtype: float64

In [16]:
comparison_df[comparison_df["count_sustained"]>0]["count_sustained"].describe()

count    240.000000
mean       1.041667
std        0.238399
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        3.000000
Name: count_sustained, dtype: float64

In [17]:
comparison_df[comparison_df["count_sustained"]>0].groupby("count_sustained").size()

count_sustained
1.0    232
2.0      6
3.0      2
dtype: int64

In [19]:
comparison_df[target_col].sum()

1002